# INTRODUCTION / BUSINESS PROBLEM:

* TO analysis the toronto neighbour data and based on that data, we need to find out if any person is getting job offer in that paricular location so whether that person should move to that location or not ? 

# SOURCE DATA : Wikipedia 
https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
To provide the stakeholders the necessary information about data :

# DATA COLLECTION 

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
req = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(req.content,'lxml')
table = soup.find_all('table')[0]
df = pd.read_html(str(table))
neighborhood=pd.DataFrame(df[0])

In [2]:
neighborhood.head()

,0,1,2
0,Postal code,Borough,Neighborhood
1,M1A,Not assigned,NaN
2,M2A,Not assigned,NaN
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village


In [3]:
neighborhood.shape

(181, 3)

In [4]:
neighborhood.columns =['Postal code', 'Borough','Neighborhood']

In [5]:
neighborhood.head()

,Postal code,Borough,Neighborhood
0,Postal code,Borough,Neighborhood
1,M1A,Not assigned,NaN
2,M2A,Not assigned,NaN
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village


In [6]:
neighborhood.shape

(181, 3)

In [7]:
toronto_df=neighborhood.drop([neighborhood.index[0]])

In [8]:
toronto_df.head()

,Postal code,Borough,Neighborhood
1,M1A,Not assigned,NaN
2,M2A,Not assigned,NaN
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Regent Park / Harbourfront


In [9]:
toronto_df.shape

(180, 3)

In [10]:
toronto_df.drop(toronto_df.index[toronto_df['Borough'] =="Not assigned"], inplace = True)

In [11]:
toronto_df.shape

(103, 3)

In [12]:
toronto_df.head()

,Postal code,Borough,Neighborhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Regent Park / Harbourfront
6,M6A,North York,Lawrence Manor / Lawrence Heights
7,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


# METHODOLOGY:

For this report I will use a k cluster method that could help to new joiner to shift that particular location is good or not 


END OF PART 1

In [13]:
geodf=pd.read_csv("Geospatial_Coordinates.csv")

In [14]:
geodf.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [15]:
geodf.rename({'Postal Code': 'Postal code', 'Latitude': 'Latitude','Longitude':'Longitude'}, axis=1, inplace=True)

In [16]:
geodf.head()

,Postal code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [17]:
result = pd.merge(toronto_df, geodf)

In [18]:
result.head()

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636
3,M6A,North York,Lawrence Manor / Lawrence Heights,43.718518,-79.464763
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.662301,-79.389494


In [20]:
from geopy.geocoders import Nominatim

In [21]:
import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as colors

In [23]:
from sklearn.cluster import KMeans
import folium

In [24]:
result1=result.iloc[:,1:]

In [25]:
result1.head()

,Borough,Neighborhood,Latitude,Longitude
0,North York,Parkwoods,43.753259,-79.329656
1,North York,Victoria Village,43.725882,-79.315572
2,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636
3,North York,Lawrence Manor / Lawrence Heights,43.718518,-79.464763
4,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.662301,-79.389494


In [26]:
result1.shape

(103, 4)

In [28]:
address = 'North York, NY'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of North York are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of North York are 40.741235950000004, -73.35669116536101.


In [29]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(result1['Latitude'], result1['Longitude'], result1['Borough'], result1['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork


In [30]:
CLIENT_ID = 'JAF1W505F51KJWCELTECRVWBLH1WGV5YUS2K0QUSM2RQI15B'
CLIENT_SECRET = 'NA5JBMTQH3CZVRU0D5YNPBUWBWFPXLWO1DDZ0V34SDUHM0LO'
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: JAF1W505F51KJWCELTECRVWBLH1WGV5YUS2K0QUSM2RQI15B
CLIENT_SECRET:NA5JBMTQH3CZVRU0D5YNPBUWBWFPXLWO1DDZ0V34SDUHM0LO


In [33]:
North_York_data = result1[result1['Borough'] == 'North York'].reset_index(drop=True)
North_York_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,North York,Parkwoods,43.753259,-79.329656
1,North York,Victoria Village,43.725882,-79.315572
2,North York,Lawrence Manor / Lawrence Heights,43.718518,-79.464763
3,North York,Don Mills,43.745906,-79.352188
4,North York,Glencairn,43.709577,-79.445073


In [35]:
North_York_data.loc[0, 'Neighborhood']

neighborhood_latitude = North_York_data.loc[0, 'Latitude'] 
neighborhood_longitude = North_York_data.loc[0, 'Longitude']

neighborhood_name = North_York_data.loc[0, 'Neighborhood']

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Parkwoods are 43.7532586, -79.3296565.


In [36]:
North_York_data.loc[0, 'Neighborhood']

'Parkwoods'

In [37]:
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL


'https://api.foursquare.com/v2/venues/explore?&client_id=JAF1W505F51KJWCELTECRVWBLH1WGV5YUS2K0QUSM2RQI15B&client_secret=NA5JBMTQH3CZVRU0D5YNPBUWBWFPXLWO1DDZ0V34SDUHM0LO&v=20180605&ll=43.7532586,-79.3296565&radius=500&limit=100'

In [38]:
import json

In [39]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e894fe1da9e14001b2fdc72'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Parkwoods - Donalda',
  'headerFullLocation': 'Parkwoods - Donalda, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 2,
  'suggestedBounds': {'ne': {'lat': 43.757758604500005,
    'lng': -79.32343823984928},
   'sw': {'lat': 43.7487585955, 'lng': -79.33587476015072}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4e8d9dcdd5fbbbb6b3003c7b',
       'name': 'Brookbanks Park',
       'location': {'address': 'Toronto',
        'lat': 43.751976046055574,
        'lng': -79.33214044722958,
        'labeledLatLngs': 

In [40]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [42]:
from pandas.io.json import json_normalize

In [43]:

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Brookbanks Park,Park,43.751976,-79.332140
1,Variety Store,Food & Drink Shop,43.751974,-79.333114


In [44]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

2 venues were returned by Foursquare.


# Based on nearby store and park , new joiner will decide whethere he/she has to move that place or not